In [33]:
#FLANN Algorithm Test
import numpy as np
import cv2
import os, os.path
import matplotlib.pyplot as plt

#Load Test Images
imageDir = "test/" #specify your path here
image_path_list = []
test_images = []
test_labels = []
valid_image_extensions = [".jpg", ".jpeg", ".png", ".tif", ".tiff"] #specify your vald extensions here
valid_image_extensions = [item.lower() for item in valid_image_extensions]

for file in os.listdir(imageDir):
    extension = os.path.splitext(file)[1]
    if extension.lower() not in valid_image_extensions:
        continue
    image_path_list.append(os.path.join(imageDir, file))

for imagePath in image_path_list:
    test_labels.append(imagePath)
    test_images.append(cv2.cvtColor(cv2.imread(imagePath), cv2.COLOR_BGR2GRAY))

In [34]:
#Get Keypoints and Descriptors From Train Images
imageDir = "train/" #specify your path here
image_path_list = []
train_images = []
train_labels = []
valid_image_extensions = [".jpg", ".jpeg", ".png", ".tif", ".tiff"] #specify your vald extensions here
valid_image_extensions = [item.lower() for item in valid_image_extensions]

for file in os.listdir(imageDir):
    extension = os.path.splitext(file)[1]
    if extension.lower() not in valid_image_extensions:
        continue
    image_path_list.append(os.path.join(imageDir, file))

for imagePath in image_path_list:
    train_labels.append(imagePath)
    train_images.append(cv2.cvtColor(cv2.imread(imagePath), cv2.COLOR_BGR2GRAY))

In [35]:
for label in train_labels:
    print(label)
    
train_labels_text = ["100EurB", "100EurF","10EurB", "10EurF", "200EurB", "200EurF", "20EurB", "20EurF", "500EurB", "500EurF",
                "50EurB", "50EurF", "5EurB", "5Eur2002B", "5EurF", "5Eur2002F"]

import pickle
with open('train_labels.pkl', 'wb') as f:
    pickle.dump(train_labels, f)

train/100eu_back.jpg
train/100eu_front.jpg
train/10eu_back.jpg
train/10eu_front.jpg
train/200eu_back.jpg
train/200eu_front.jpg
train/20eu_back.jpg
train/20eu_front.jpg
train/500eu_back.jpg
train/500eu_front.jpg
train/50eu_back.jpg
train/50eu_front.jpg
train/5eu_back.jpg
train/5eu_back_2002.jpg
train/5eu_front.jpg
train/5eu_front_2002.jpg


In [124]:
#Get Keypoints and Descriptors For Each Train Image
sift = cv2.xfeatures2d.SIFT_create()
#surf = cv2.xfeatures2d.SURF_create(400)
descriptors = []

for image in train_images:
    kp, des = sift.detectAndCompute(image, None)
    descriptors.append(des)

In [52]:
#Save Descriptors List so we can use them after
import pickle
with open('descriptors_surf.pkl', 'wb') as f:
    pickle.dump(descriptors, f)

#Load Descriptors List
#import pickle
#descriptors = pickle.load(open('descriptors.pkl', 'rb'))

In [79]:
#Let's test clustering for bag of visual words
vStack = np.array(descriptors[0])
for remaining in descriptors[1:]:
    vStack = np.vstack((vStack, remaining))
    descriptor_vstack = vStack.copy()

#from sklearn.cluster import KMeans
#X = np.array(descriptor_vstack)
#kmeans = KMeans(n_clusters=9, random_state=0)
#kmeans.fit(X)

21059

In [54]:
#Let's get each image as an image clustered words
image_clustered_words = []
for descriptor in descriptors:
    image_clustered_words.append(kmeans.predict(descriptor))
    

In [55]:
#Finally make a histogram of clustered word counts for each image. These are the final features.
img_bow_hist = np.array(
[np.bincount(clustered_words, minlength=9) for clustered_words in image_clustered_words])

In [97]:
#Encode labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels_text)
le.classes_
train_labels_encoded = le.transform(train_labels_text)
#le.inverse_transform(train_labels_encoded)

print(train_labels_encoded)

[ 0  1  2  3  4  5  6  7  8  9 10 11 14 12 15 13]


In [109]:
count = 0
extended_train_labels = []
i = 0
while i < len(descriptors):
    while (count < len(descriptors[i])):
        extended_train_labels.append(train_labels_encoded[i])
        count = count + 1
    i = i + 1
    count = 0
print(np.array(extended_train_labels).shape, descriptor_vstack.shape)       

(537750,) (537750, 64)


In [111]:
#Let's Use An SVM To Train Model
from sklearn.svm import LinearSVC
SVM = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
SVM.fit(descriptor_vstack, extended_train_labels)

SVM.score(X=descriptor_vstack, y=extended_train_labels)


0.17462017666201768

In [123]:
#Let's test on a new image
from collections import Counter
for img in test_images:
    #sift = cv2.xfeatures2d.SIFT_create()
    surf = cv2.xfeatures2d.SURF_create(400)
    kp, desc = surf.detectAndCompute(img, None)

    #clustered_desc = kmeans.predict(desc)
    #img_bow_hist_img = np.bincount(clustered_desc, minlength=9)

    # reshape to an array containing 1 array: array[[1,2,3]]
    # to make sklearn happy (it doesn't like 1d arrays as data!)
    #test_img = desc.reshape(1,-1)
    prediction = SVM.predict(desc)
    print("Prediction is: ", max(Counter(le.inverse_transform(prediction))))

Prediction is:  50EurF
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  50EurF
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction is:  5Eur2002F
Prediction is:  5Eur2002F
Prediction is:  50EurF
Prediction 

In [45]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('test/100-100.jpg',0)          # queryImage
#img2 = cv2.imread('500eu_front.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
#kp2, des2 = sift.detectAndCompute(img2,None)
matches = []

for i in range(len(descriptors)):
    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=100)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params,search_params)

    matches.append(flann.knnMatch(des1,descriptors[i],k=2))

# Need to draw only good matches, so create a mask
#matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
#for i,(m,n) in enumerate(matches):
    #if m.distance < 0.7*n.distance:
        #matchesMask[i]=[1,0]

#draw_params = dict(matchColor = (0,255,0),
                   #singlePointColor = (255,0,0),
                   #matchesMask = matchesMask,
                   #flags = 0)

#img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)

#plt.imshow(img3,),plt.show()

In [46]:
lenghts = []
for match in matches:
    lenghts.append(len(match))
    

In [47]:
for i in range(len(lenghts)):
    if lenghts[i] == max(lenghts):
        prediction = train_labels[i]
print(prediction)

train/5eu_front_2002.jpg


In [20]:
#index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#index_params= dict(algorithm = FLANN_INDEX_LSH,
                   #table_number = 6, # 12
                   #key_size = 12,     # 20
                   #multi_probe_level = 1) #2

import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('5.JPG',0)          # queryImage
img2 = cv2.imread('500eu_front.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=100)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches_5eur = flann.knnMatch(des1,des2,k=5)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches_5eur))]

# ratio test as per Lowe's paper
#for i,(m,n) in enumerate(matches_5eur):
    #if m.distance < 0.7*n.distance:
        #matchesMask[i]=[1,0]

#draw_params = dict(matchColor = (0,255,0),
                   #singlePointColor = (255,0,0),
                   #matchesMask = matchesMask,
                   #flags = 0)

#img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches_5eur,None,**draw_params)

#plt.imshow(img3,),plt.show()

In [25]:
#index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#index_params= dict(algorithm = FLANN_INDEX_LSH,
                   #table_number = 6, # 12
                   #key_size = 12,     # 20
                   #multi_probe_level = 1) #2

import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('20__(4).JPG',0)          # queryImage
img2 = cv2.imread('500eu_front.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=100)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches_100eur = flann.knnMatch(des1,des2,k=5)

# Need to draw only good matches, so create a mask
#matchesMask = [[0,0] for i in range(len(matches_100eur))]

# ratio test as per Lowe's paper
#for i,(m,n) in enumerate(matches_100eur):
    #if m.distance < 0.7*n.distance:
        #matchesMask[i]=[1,0]

#draw_params = dict(matchColor = (0,255,0),
                   #singlePointColor = (255,0,0),
                   #matchesMask = matchesMask,
                   #flags = 0)

#img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches_100eur,None,**draw_params)

#plt.imshow(img3,),plt.show()

In [26]:
print(len(matches), len(matches_5eur), len(matches_100eur))

531 313 479


In [155]:
# create BFMatcher object
bf = cv2.BFMatcher_create()
#bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
kp, des2 = sift.detectAndCompute(test_images[15], None)
print("Reading... ", test_labels[15])

# Match descriptors.
matches = []
for descriptor in descriptors:
    matches.append(bf.knnMatch(descriptor,des2, k=2))

# Apply ratio test
#good = []
#for m,n in matches:
    #if m.distance < 0.75*n.distance:
        #good.append([m])

Reading...  test/10__(8).jpg


In [156]:
mat_len = []
for i in range(len(matches)):
    mat_len.append(len(matches[i]))

for i in range(len(mat_len)):
    if mat_len[i] == max(mat_len):
        print(mat_len[i], max(mat_len))
        print("Prediction is ", train_labels_text[i])

338540 338540
Prediction is  500EurB


194725